In [1]:
import os
import ee
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
import time
num_seed=30
#random.seed(num_seed)

In [2]:
ee.Initialize()


In [3]:
#Define functions for mapping MapBiomas and simplifying the legend
coverage_palette =  ['ffffff', '129912', '1f4423', '006400', '00ff00', '687537', '76a5af', '29eee4', 
                     '77a605', '935132', 'bbfcac', '45c2a5', 'b8af4f', 'f1c232', 'ffffb2', 'ffd966', 
                     'f6b26b', 'f99f40', 'e974ed', 'd5a6bd', 'c27ba0', 'fff3bf', 'ea9999', 'dd7e6b', 
                     'aa0000', 'ff99ff', '0000ff', 'd5d5e5', 'dd497f', 'b2ae7c', 'af2a2a', '8a2be2', 
                     '968c46', '0000ff', '4fd3ff']


simple_palette = ['129912','BBFCAC','FFFFB2','EA9999','0000FF','D5D5E5']
statesViz = {'min': 0, 'max': 34, 'palette': coverage_palette};
simpleStatesViz = {'min': 1, 'max': 6, 'palette': simple_palette};

change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 0, 'max': 1, 'palette': change_detection_palette};

#Load in mapbiomas
mapbiomas_states=ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1')
states_mask = mapbiomas_states.mask()
projection_30m = mapbiomas_states.projection()

#Define function to convert hierarchical legend to simplest form
def simplify_legend(bandName):
    simplify = mapbiomas_states.expression(
        '(b0 >=1)  && (b0<10) ? 1 :'+
        '((b0>=10) && (b0<14)) || (b0==32) || (b0==29) ? 2 :'+
        '((b0>=18) && (b0<22)) || ((b0>=14)&&(b0<16)) ? 3 :'+
        '((b0>=22) && (b0<26)) || (b0==30) ? 4 :'+
        '(b0==26) || (b0==33) || (b0==31) ? 5 : 6', 
        {
          'b0': mapbiomas_states.select([bandName])
        })
    simplify = simplify.select(['constant'],[bandName])
    return simplify

#Bands we are interested in
bandList = ['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 
             'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 
             'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 
             'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 
             'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 
             'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 
             'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 
             'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 
             'classification_2017', 'classification_2018']
bandsEEList = ee.List(bandList) 
states_simple = ee.ImageCollection(bandsEEList.map(simplify_legend)).toBands()
states_simple = states_simple.select(states_simple.bandNames(),bandsEEList)
states_simple = states_simple.updateMask(states_mask)

#Map
Map1 = geemap.Map(center=[-9,-51], zoom=4)
Map1.addLayer(mapbiomas_states.select('classification_2018'),statesViz,name='Original MapBiomas')
Map1.addLayer(states_simple.select('classification_2018'),simpleStatesViz,name='Simplified MapBiomas')
display(Map1)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [4]:
#Convert long band names to short band names
intBandNames = ['1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', 
             '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',  '2005', '2006', '2007', '2008', 
             '2009', '2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018']

int_states_simple = states_simple.select(bandList,intBandNames)

def convert_to_image_collection(bandName):
    return states_simple.select(bandName)


In [5]:
#Define functions for finding different types of change in the MapBiomas series
def lc_one_change(bandName):
    '''
    Determines if there was one change occurance from year i to year i+1. Returns an image with values:
    1 if state(i) != state(i+1)
    0 if state(i) == state(i+1)
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    return int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))

def lc_consistent_change_one_year(bandName):
    '''
    Determines if change that occured from i to i+1 stayed in i+2
    1 if state(i) != state(i+1) and state(i+1) == state(i+2)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    stayed = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextYearString]))
    lc_consistently_changed = changed.bitwise_and(stayed)
    return lc_consistently_changed

def lc_consistent_change_two_years(bandName):
    '''
    Determines if change that occured from i to i+1 stayed in i+2 and i+3
    1 if state(i) != state(i+1) and state(i+1) == state(i+2) and state(i+1) == state(i+3)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    nextNextNextYear = currentYear.add(ee.Number(3))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    nextNextNextYearString = ee.List(intBandNames).get(nextNextNextYear)
    
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    stayed_one_year = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextYearString]))
    stayed_two_year = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextNextYearString]))
    lc_consistently_changed = changed.bitwise_and(stayed_one_year.bitwise_and(stayed_two_year))
    return lc_consistently_changed


In [6]:
#Apply functions to MapBiomas image series
years_for_one_change = np.arange(0,len(intBandNames)-1).tolist()#
years_for_two_change = np.arange(0,len(intBandNames)-2).tolist()#
years_for_three_change = np.arange(0,len(intBandNames)-3).tolist()#

states_lc_one_change = ee.ImageCollection(ee.List(years_for_one_change).map(lc_one_change)).toBands()
states_lc_one_change = states_lc_one_change.select(states_lc_one_change.bandNames(),intBandNames[0:len(intBandNames)-1])

# states_lc_consistent_change_one_year = ee.ImageCollection(ee.List(years_for_two_change).map(lc_consistent_change_one_year)).toBands()
# states_lc_consistent_change_one_year = states_lc_consistent_change_one_year.select(states_lc_consistent_change_one_year.bandNames(),intBandNames[0:len(intBandNames)-2])

states_lc_consistent_change_two_years = ee.ImageCollection(ee.List(years_for_three_change).map(lc_consistent_change_two_years)).toBands()
states_lc_consistent_change_two_years = states_lc_consistent_change_two_years.select(states_lc_consistent_change_two_years.bandNames(),intBandNames[0:len(intBandNames)-3])



In [7]:
#Get mask of when tiles had one change, we only want to sample pixels that had at least one change
# i.e. mask pixels that had no change
states_lc_one_change_for_mask = states_lc_one_change.select(intBandNames[0:len(intBandNames)-3])
#print(intBandNames[0:len(intBandNames)-3])
change_occured = states_lc_one_change_for_mask.reduce(ee.Reducer.max())


In [8]:
#Find pixels that had at least one year of consistent change, add 1 so that 0 can be the mask/no data value
consistent_change_occurred = states_lc_consistent_change_two_years.reduce(ee.Reducer.max()).add(1)
#Update mask
consistent_change_occurred_masked = consistent_change_occurred.updateMask(change_occured)
#Rename band from "max" from reducer to "consistent_change"
consistent_change_occurred_masked = consistent_change_occurred_masked.select(['max'],['consistent_change'])
#Consistent change raster is now coded:
# 0 = no data, change did not occur in this pixel in any year
# 1 = no consistent change in this pixel in any year
# 2 = at least one year of consistent change occurred in this pixel


print(consistent_change_occurred_masked.getInfo())

#Define color palettes and map
one_change_detection_palette = ['379c4d','04e735']
oneChangeDetectionViz = {'min': 0, 'max': 1, 'palette': one_change_detection_palette};

consistent_change_detection_palette = ['df07b5','0741df']
consistentChangeDetectionViz = {'min': 1, 'max': 2, 'palette': consistent_change_detection_palette};

Map2 = geemap.Map(center=[-9,-51], zoom=4)
Map2.addLayer(change_occured.updateMask(change_occured),oneChangeDetectionViz,name='One Change')
#Light green shows one change occurred, dark green shows no change occurred (now masked so only dark green should show)
Map2.addLayer(consistent_change_occurred_masked,consistentChangeDetectionViz,name='Consistent Change')
#Pink shows there was not consistent change, blue shows consistent change
display(Map2)
#Pink is no change
#Blue is change

#Consistent change raster is now coded:
# 0 = no data, change did not occur in this pixel in any year
# 1 = no consistent change in this pixel in any year (PINK)
# 2 = at least one year of consistent change occurred in this pixel (BLUE)



{'type': 'Image', 'bands': [{'id': 'consistent_change', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [9]:

#Define function to sample points from image stratified by specified classBand, also returns 
# latitude and longitude of sampled points
def getStratifiedSampleBandPoints(image, region, bandName, **kwargs):
    new_image = image.addBands(ee.Image.pixelLonLat().reproject(image.projection()))
    dargs = {
        'numPoints': 1000,
        'classBand': bandName,
        'projection': new_image.projection().crs().getInfo(),
        'scale': image.projection().nominalScale().getInfo(),
        'region': region.geometry()
    }
    dargs.update(kwargs)
    print(dargs)
    stratified_sample = new_image.stratifiedSample(**dargs)
    return stratified_sample

#Function to convert feature collection to pandas dataframe
def get_dataframe_from_feature_collection(feature_collection, property_names):
    df = pd.DataFrame()
    for property_name in property_names:
        property_values = feature_collection.aggregate_array(property_name).getInfo()
        df[property_name] = property_values
    return df

#Function to convert pandas dataframe to feature collection
def convert_points_df_to_feature_collection(df,projection='EPSG:4326',lat_name='latitude',lon_name='longitude'):
    feature_collection_list = []
    for i,row in df.iterrows():
        geometry = ee.Geometry.Point([row[lon_name],row[lat_name]],projection)
        row_dict = row.to_dict()
        row_feature = ee.Feature(geometry,row_dict)
        feature_collection_list.append(row_feature)
    return ee.FeatureCollection(feature_collection_list)


def sampleBandDataToList(sampleData):
    return ee.FeatureCollection(sampleData).flatten().aggregate_array('first')

def getSampleImageData(image, sampleBandPoints):
    sampleImageData = image.reduceRegions(
            collection=sampleBandPoints,
            reducer=ee.Reducer.first()
            )
    return sampleImageData.aggregate_array('first')



def convert_feature_collection_to_list(feature_collection):
    return feature_collection.toList(feature_collection.size().getInfo())





In [11]:
#Load in feature collection to sample over
Brazil_adm1 = ee.FeatureCollection('users/listerkristineanne/DynamicWorld/ChangeDetection/MapBiomas/Brazil_adm1')
Brazil_adm0 = ee.FeatureCollection('users/listerkristineanne/DynamicWorld/ChangeDetection/MapBiomas/Brazil_adm0')

#selected_adm1 = Brazil_adm1.filterMetadata('GID_1','equals','BRA.12_1')
#Reproject consistent change image to the original projection, Earth Engine will not force this calculation
#until we do so
consistent_change_occurred_masked_reprj = consistent_change_occurred_masked.reproject(mapbiomas_states.projection())

#Sample points
sample_points = getStratifiedSampleBandPoints(consistent_change_occurred_masked_reprj,Brazil_adm0,
                                     'consistent_change',numPoints=10000)


{'numPoints': 10000, 'classBand': 'consistent_change', 'projection': 'EPSG:4326', 'scale': 29.999999999999996, 'region': <ee.geometry.Geometry object at 0x12985a160>}


In [12]:
##At higher resolution, the calculation times out to print to the client side,
##So we will export it the google drive of the current user
export_sample_points_task = ee.batch.Export.table.toDrive(
    collection=sample_points, 
    description = "SamplePoints_10K", 
    fileNamePrefix = 'SamplePoints_10K')

export_sample_points_task.start()
print(export_sample_points_task)


<Task EXPORT_FEATURES: SamplePoints_10K (UNSUBMITTED)>


Once task export is complete, download csv from google drive to pull in below

In [ ]:

training_points = pd.read_csv('/Users/kristine/Downloads/Sample_Points_1000.csv')
columns_to_remove = ['system:index','.geo']
columns_to_keep = [x for x in list(training_points) if x not in columns_to_remove]
training_points = training_points[columns_to_keep]
training_points_fc = convert_points_df_to_feature_collection(training_points)
#Map results
consistent_change_detection_palette = ['df07b5','0741df']
consistentChangeDetectionViz = {'min': 1, 'max': 2, 'palette': consistent_change_detection_palette};
Map3 = geemap.Map(center=[-9,-51], zoom=4)
Map3.addLayer(consistent_change_occurred_masked,consistentChangeDetectionViz,name='Consistent Change')
Map3.addLayer(training_points_fc,name='Sampled Points')
#Pink shows there was not consistent change, blue shows consistent change
display(Map3)


In [ ]:
full_res = {}

for i in years_for_three_change:
    bandName = intBandNames[i]
    res = ee.Dictionary({
    'one_change_'+bandName: getSampleImageData(states_lc_one_change.select([bandName]).reproject(projection_30m), training_points_fc),
    'consistent_change_two_years_'+bandName: getSampleImageData(states_lc_consistent_change_two_years.select([bandName]).reproject(projection_30m), training_points_fc),
    }).getInfo()
    full_res.update(res)
    
ordered = ['one_change','consistent_change_two_years']


band_sample_df = pd.DataFrame(full_res)
band_sample_df


In [ ]:

def get_years_of_consistent_change(df,column_names,years):
    for column_name in column_names:
        df[column_name] = ''
        match_columns = [x for x in list(df) if column_name in x]
        for i,row in df.iterrows():
            row = row[match_columns]
            positive_columns = [x for x in match_columns if row[x]==1]
            if len(positive_columns) == 0:
                df.at[i,column_name] = None
            else:
                years_of_change = [x for x in years if str(x) in str(positive_columns)]
                years_of_change = ' '.join([str(item) for item in years_of_change])
                df.at[i,column_name] = years_of_change
            
    return df

year_names = [intBandNames[x] for x in years_for_three_change]
collapsed_df = get_years_of_consistent_change(band_sample_df,ordered,year_names)
collapsed_df = collapsed_df[ordered]
merged_collapsed_df = pd.concat([training_points,collapsed_df],axis=1)
merged_collapsed_df.to_csv('/Users/kristine/Downloads/NewSampledPoints_wide.csv',index=False)
merged_collapsed_df

In [ ]:
start_time = time.time()
#out_df = pd.DataFrame(columns=['consistent_change','year','latitude','longitude'])
#Pull a random year
out_df = merged_collapsed_df.copy()
for i,row in out_df.iterrows():
    #print(row)
    if row['consistent_change_two_years'] is not None:
        out_df.at[i,'consistent_change'] = 1
        years = row['consistent_change_two_years'].split(' ')
        out_df.at[i,'year'] = random.choice(years)
            
    else:
        out_df.at[i,'consistent_change'] = 0
        years = row['one_change'].split(' ')
        out_df.at[i,'year'] = random.choice(years)

out_df = out_df[['consistent_change','latitude','longitude','year']]
end_time = time.time()
print(end_time-start_time)
out_df

In [ ]:
out_df.to_csv('/Users/kristine/Downloads/NewSampledPoints_wchange.csv',index=False)


In [ ]:
#In the neighborhood:
#How many pixels changed
#How many changed back
#diversity of categories of the neighborhood

#imagery?